# IA conversacional – ¡también conocido como Chatbot!

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Inicializar

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [ ]:
system_message = "Eres un assistente muy útil"

# La estructura interna del historial de mensajes

Originalmente, gradio esperaba recibir una función llamada:

`chat(message, history)`

La cual debía recibir `history` en un formato particular, que debemos asignar al formato OpenAI antes de llamar a OpenAI:

```
[
{"role": "system", "content": "system message here"},
{"role": "user", "content": "first user prompt here"},
{"role": "assistant", "content": "the assistant's response"},
{"role": "user", "content": "the new user prompt"},
]
```

¡Pero Gradio se ha actualizado! Ahora pasará `history` en el formato exacto de OpenAI, perfecto para que lo enviemos directamente a OpenAI.

¡Así que nuestro trabajo se volvió más fácil!

Escribiremos una función `chat(message, history)` donde:
**message** es el mensaje que se debe usar
**history** es la conversación anterior, en formato OpenAI

Combinaremos el mensaje del sistema, el historial y el último mensaje, y luego llamaremos a OpenAI.

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("El historial es:")
    print(history)
    print("Y los mensajes son:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## ¡Y entonces entra la magia de Gradio!

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message = """Eres un asistente útil en una tienda de ropa.
Debes tratar de alentar gentilmente al cliente a que pruebe los artículos que están en oferta.
Los sombreros tienen un 60 % de descuento y la mayoría de los demás artículos tienen un 50 % de descuento.
Por ejemplo, si el cliente dice 'Quiero comprar un sombrero',
podrías responder algo como 'Genial, tenemos muchos sombreros, incluidos varios que son parte de nuestro evento de rebajas'.
Anima al cliente a comprar sombreros si no está seguro de qué comprar."""

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inline=False)

In [ ]:
system_message += "\nSi el cliente pide zapatos, debes responder que los zapatos no están en oferta hoy, \
¡pero recuérdale al cliente que mire los sombreros!"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inline=False)

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    if 'cinturon' in message:
        messages.append({"role": "system", "content": "Para mayor contexto, la tienda no vende cinturones,\
        pero asegúrate de señalar otros artículos en oferta."})
    
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inline=False)

## Ollama

In [ ]:
import ollama


def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat(model="llama3.2", messages=messages)
    return response['message']['content']

gr.ChatInterface(fn=chat, type="messages").launch(inline=False)